# Title: Mise en Prep: Restraunt Supply Chain Manager

Mathan Liu | liu02375@umn.edu

### Description
Mise en Prep is an application designed to help chefs and restraunts manage their food supply chains everyday. Restraunts need to ensure they get their ingredients on time so they are able to prepare them in time for service. This MVP uses local grocery stores locations and finds the average time it takes for deliveries to get from the store to the restraunt. 

## Specs

| Value | Effort  | Feature    | 
|:------|:--------|:-----------|
| H     | L       | Plot all grocery store locations as points on a map |
| H     | M       | Visualize the optimized route |
| H     | M       | Find the time it takes for each route|
| H     | M       | Pass through various sets of supplier data locations|
| H     | L       | Input different restraunts|
| H     | L       | Display the contact information for each restaurant in case of emergencies|

# The Code!

### Packages
In this code call are all of the packages I used. The primary one I used was the openrouteservice which requires an API key to map things on a road. I took out my own API key but the code should work with your own API key. As a sidenote, in case this comes up, it is very quick and easy to get a token for the ors API, all you need is to make an account in the ORS dev dashboard, redeem a token, put it in the code, and everything should work. As for the dataset I used, it is a trimmed down dataset, to accomadate for ors, from data.gov. I have uploaded the data set with my MVP and will put the address of the restraunt I used so if you want you can test out the application MVP yourself. Thank you!

### Functions
I hope the commenting is clear and the functions all make sense!

### Main
At the very bottom of the code we are running the code with the main function. 

In [119]:
#Import Packages
import folium
import pandas as pd
from geopy.geocoders import Nominatim
import openrouteservice as ors
client=ors.Client(key='#Your own key')
import ipywidgets as widgets
from prettytable import PrettyTable
#I tried very hard to figure out how to create a comprehensive dashboard with all of my information but kept getting errors. I just left some of the packages I was trying to use to show my work process a bit. 
from bokeh.plotting import figure, show, output_notebook
from IPython.display import display, clear_output

#Creating the Map
def create_map():
    m = folium.Map(location=list(reversed([-90.071234696353, 29.938076735378])), zoom_start=14)
#for the input all you have to type in is Grocery_Stores.csv
def supplier_data():
    #Input/Clean/Plot of Supplier Data
    file_path = input("Input the location of all of your suppliers are in a .csv format: ")
    df = pd.read_csv(file_path)
    #clean the data 
    df['Latitude'] = df['the_geom'].str.extract(r'POINT \(([^ ]+) ').astype(float)
    df['Longitude'] = df['the_geom'].str.extract(r' ([^ ]+)\)').astype(float)
    coords = []
    #here we take the coords and plot them as points on the map
    for index, row in df.iterrows():
        coords.append((row['Latitude'], row['Longitude']))
    for coords in coords:
        folium.Marker(location=list((coords))).add_to(m)
#Here is the address for the restaurant I used for all my testing: 600 Carondelet St, New Orleans, LA 70130
def restraunt_add():
    #Restraunt Address
    address = input("Enter the address: ")
    geolocator = Nominatim(user_agent="address_plotter")
    #using geolocator so all you have to do is put in an address and finds its coords
    location = geolocator.geocode(address)
    if location is None:
        print("Location not found for the given address.")
        #plotting an orange icon for the restaurant address
    orange_icon = folium.Icon(color='orange')    
    folium.Marker([location.latitude, location.longitude], popup=address, icon=orange_icon).add_to(m)

#ORS Route Optimization
def ors_calculations():
    coords = [(float(row['Longitude']), float(row['Latitude'])) for index, row in df.iterrows()]
    delivery_end = address
    #I had to reverse my address coordinates but got really lost and couldn't figure out a fix for about an hour, so I just put the coords flip flopped so I could work on other aspects of the project. I will fix in the final hopefully. 
    
    #vehicles represent the mode of transport going to and from the restraunt to stores
    vehicles = [ors.optimization.Vehicle(id=0, profile='driving-car', start=(-90.071234696353, 29.938076735378), end=None, capacity=[10])]
    
    #jobs are basically where the stores are 
    jobs = [ors.optimization.Job(id=index, location=(coords[1], coords[0])) for index, coords in enumerate(coords)]
    
    #optimization                             
    optimized = client.optimization(jobs=jobs, vehicles=vehicles, geometry=True)
    #plotting purple lines over the optimized routes on the road
    line_colors = ['purple']
    for route in optimized['routes']:
        folium.PolyLine(
            locations=[list(reversed(coords)) for coords in ors.convert.decode_polyline(route['geometry'])['coordinates']],
            color=line_colors[route['vehicle']]
        ).add_to(m)
def route_times():
#IMPORTANT: I was using a for loop to loop through all the end coordinates to get the distance between every single store and the restaurant but the distance got maxed out and kept error coding me so I just am demonstrating with one store.
#This case is from Read Seafood to the restraunt, the end coord is Read Seafood
    start_coords = (-90.071234696353, 29.938076735378)
    end_coord = (-89.980786717109, 30.042348592751)
    route = client.directions(
        coordinates=[start_coords, end_coord],
        profile='driving-car', 
        format='geojson' 
        )
    #here we are able to get the road distance from using the power of ors
    road_distance = route['features'][0]['properties']['segments'][0]['distance']
    #now we want to take the distance to find the time
    #Time it takes to travel assuming car going around 30mph
    #Road distance calculated is currently in meters, must translate into miles
    rd_miles= road_distance/1609.34
    travel_time=rd_miles/30
    #translate into minutes
    tt_inmins=travel_time*60
    print("The time to get to Read Seafood is", tt_inmins,"minutes.")
#here is a function for the information table 
def info_table():
    #sort which columns I want
    selected_columns = ['Business Name', 'Address', 'Phone Number']
    df_selected = df[selected_columns]
    table = PrettyTable()
    #make the table cleaner for viewing
    table.field_names = selected_columns
    for index, row in df_selected.iterrows():
        table.add_row(row)
    print(table)

    
#here I tried to make a dropdown to make it so that it would give the time from the restaurant to each individual supplier location and then also just only plot that line on the map but after spending hours on this I could not figure it out and kept getting errors so here is what remains of my work. 
def restraunts_dropdown():
    dropdown = widgets.Dropdown(
    options=df['Business Name'],
    description='Select an option:',
    disabled=False,
    )
    #display the dropdown widget
    display(dropdown)
    
    def on_dropdown_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            selected_business = change['new']
            geom_value = df.loc[df['Business Name'] == selected_business, 'the_geom'].iloc[0]
            print(f"The geometry value for {selected_business} is: {geom_value}")
    dropdown.observe(on_dropdown_change, names='value')



#Here is our main function where we compiled everything!
def main():
    create_map()
    supplier_data()
    restraunt_add()
    ors_calculations()
    route_times()
    restraunts_dropdown()
    info_table()
    
#here we run the whole program     
if __name__ == "__main__":
    main()        
m


KeyboardInterrupt: Interrupted by user

## Research/Project Story
I feel like I didn't really take a classic approach towards this project as my final ended up not having a lot of data or graphs. I am super interested in startups and entrepreneurship so I wanted to make an application that I thought would be an interesting startup idea that tied in the GIS information we learned in this course. Thus, I created this restaurant supply chain manager. My one map is the main aspect of the product and gives the chefs a visual look at where and how far away their suppliers are. The one table is simply extracted from the CSV and made to look nice so that it looks like somewhat of a coherent dashboard. It also serves a purpose as a part of the application as I believe it can help chefs reach out to and contact their suppliers when there are large delays and things of that nature. Finally, there is the most important part of the application which the little print out which states how much time it takes for each respective supplier to get to the restaurant. Using openrouteservice it maxed out my distance I could measure so I had to just use one supplier as an example, but I think it shows the viabily of my product here. Anyways I know this is a bit untraditional but I hope you enjoy and see the value in my project!